<a target="_blank" href="https://colab.research.google.com/github/sonder-art/fdd_o23/blob/main/codigo/polars/02_sintaxis_polars.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

In [ ]:
#!pip isntall polars

In [5]:
import polars as pl

In [6]:
# Crear un DataFrame simple
df = pl.DataFrame({
    "nombre": ["Ana", "Luis", "Marta"],
    "edad": [25, 32, 18]
})

# pl.col

In [7]:
# Seleccionar la columna 'edad' y realizar una operación
edad_mas_diez = df.select(
    pl.col("edad") + 10
)

print(edad_mas_diez)


shape: (3, 1)
┌──────┐
│ edad │
│ ---  │
│ i64  │
╞══════╡
│ 35   │
│ 42   │
│ 28   │
└──────┘


Ahora crea una nueva columna que aparezca en el mismo `df` que se llame `super_edad` y sea la edad al cuadrado. Tienes que usar `with_columns`, `pl.col` y solo usar una linea de codigo

In [10]:
# TU codigo aquí
df.with_columns((pl.col('edad')+10).alias("super_edad"))

nombre,edad,super_edad
str,i64,i64
"""Ana""",25,35
"""Luis""",32,42
"""Marta""",18,28


# Varias Operaciones

In [14]:
# Añadir dos nuevas columnas utilizando operaciones con 'pl.col'
a=[
    (pl.col("edad") * 2).alias("edad_doble"),
    (pl.col("edad") > 30).alias("mayor_de_30")
]
df = df.with_columns(a)

print(df)


shape: (3, 4)
┌────────┬──────┬────────────┬─────────────┐
│ nombre ┆ edad ┆ edad_doble ┆ mayor_de_30 │
│ ---    ┆ ---  ┆ ---        ┆ ---         │
│ str    ┆ i64  ┆ i64        ┆ bool        │
╞════════╪══════╪════════════╪═════════════╡
│ Ana    ┆ 25   ┆ 50         ┆ false       │
│ Luis   ┆ 32   ┆ 64         ┆ true        │
│ Marta  ┆ 18   ┆ 36         ┆ false       │
└────────┴──────┴────────────┴─────────────┘


# Groupby Complejo

In [16]:
# Crear un DataFrame de ejemplo
df = pl.DataFrame({
    "departamento": ["ventas", "ventas", "IT", "IT", "HR", "HR"],
    "empleado": ["Ana", "Luis", "Marta", "Juan", "Elena", "Carlos"],
    "salario": [30000, 35000, 42000, 40000, 38000, 32000]
})


In [18]:
df.describe

<bound method DataFrame.describe of shape: (6, 3)
┌──────────────┬──────────┬─────────┐
│ departamento ┆ empleado ┆ salario │
│ ---          ┆ ---      ┆ ---     │
│ str          ┆ str      ┆ i64     │
╞══════════════╪══════════╪═════════╡
│ ventas       ┆ Ana      ┆ 30000   │
│ ventas       ┆ Luis     ┆ 35000   │
│ IT           ┆ Marta    ┆ 42000   │
│ IT           ┆ Juan     ┆ 40000   │
│ HR           ┆ Elena    ┆ 38000   │
│ HR           ┆ Carlos   ┆ 32000   │
└──────────────┴──────────┴─────────┘>

In [17]:

# Realizar un groupby y múltiples agregaciones
resultado = df.group_by("departamento").agg([
    pl.col("salario").mean().alias("salario_promedio"),
    pl.col("salario").min().alias("salario_minimo"),
    pl.col("salario").max().alias("salario_maximo"),
    pl.col("salario").std().alias("desviacion_estandar_salario")
])

print(resultado)


shape: (3, 5)
┌──────────────┬──────────────────┬────────────────┬────────────────┬─────────────────────────────┐
│ departamento ┆ salario_promedio ┆ salario_minimo ┆ salario_maximo ┆ desviacion_estandar_salario │
│ ---          ┆ ---              ┆ ---            ┆ ---            ┆ ---                         │
│ str          ┆ f64              ┆ i64            ┆ i64            ┆ f64                         │
╞══════════════╪══════════════════╪════════════════╪════════════════╪═════════════════════════════╡
│ IT           ┆ 41000.0          ┆ 40000          ┆ 42000          ┆ 1414.213562                 │
│ ventas       ┆ 32500.0          ┆ 30000          ┆ 35000          ┆ 3535.533906                 │
│ HR           ┆ 35000.0          ┆ 32000          ┆ 38000          ┆ 4242.640687                 │
└──────────────┴──────────────────┴────────────────┴────────────────┴─────────────────────────────┘


In [20]:
# Filtrar los departamentos donde el salario promedio es mayor a 35000
deptos_alto_salario = df.group_by("departamento").agg([
    pl.col("salario").mean().alias("salario_promedio")
]).filter(pl.col("salario_promedio") > 35000)

print(deptos_alto_salario)


shape: (1, 2)
┌──────────────┬──────────────────┐
│ departamento ┆ salario_promedio │
│ ---          ┆ ---              │
│ str          ┆ f64              │
╞══════════════╪══════════════════╡
│ IT           ┆ 41000.0          │
└──────────────┴──────────────────┘
